In [1]:
import os
from tqdm import tqdm

data_dir = "/home/abhisek/Project/Language_detector_200/Ultimate_100_data_FastText"
train_file = "/home/abhisek/Project/Language_detector_200/FastText_200/train.txt"

os.makedirs(os.path.dirname(train_file), exist_ok=True)

with open(train_file, "w", encoding="utf-8") as out_file:
    for lang in tqdm(sorted(os.listdir(data_dir)), desc="Processing languages"):
        lang_dir = os.path.join(data_dir, lang)
        if os.path.isdir(lang_dir):
            for fname in os.listdir(lang_dir):
                fpath = os.path.join(lang_dir, fname)
                with open(fpath, "r", encoding="utf-8") as f:
                    text = f.read().strip().replace("\n", " ")
                    if text:
                        out_file.write(f"__label__{lang} {text}\n")


Processing languages: 100%|██████████| 200/200 [00:05<00:00, 33.95it/s]


In [2]:
import fasttext

model_path = "/home/abhisek/Project/Language_detector_200/FastText_200/lang_detect_model.bin"

model = fasttext.train_supervised(
    input=train_file,
    lr=0.7,
    epoch=50,
    wordNgrams=3,
    verbose=2,
    minCount=1,
    loss='softmax'
)

model.save_model(model_path)


Read 83M words
Number of words:  11609638
Number of labels: 200
Progress: 100.0% words/sec/thread:  191688 lr: -0.000008 avg.loss:  0.301105 ETA:   0h 0m 0s words/sec/thread:  170547 lr:  0.699267 avg.loss:  5.545167 ETA:   0h57m53sh52m33s avg.loss:  5.204518 ETA:   0h52m 3s 5.197838 ETA:   0h52m 6s  0.8% words/sec/thread:  188883 lr:  0.694174 avg.loss:  4.923356 ETA:   0h51m53s  1.5% words/sec/thread:  182466 lr:  0.689635 avg.loss:  4.985260 ETA:   0h53m21s  3.4% words/sec/thread:  190170 lr:  0.676237 avg.loss:  4.023279 ETA:   0h50m12s  3.5% words/sec/thread:  190028 lr:  0.675301 avg.loss:  3.958531 ETA:   0h50m10s 190635 lr:  0.672148 avg.loss:  3.659380 ETA:   0h49m46s% words/sec/thread:  190834 lr:  0.671224 avg.loss:  3.611973 ETA:   0h49m39s% words/sec/thread:  189694 lr:  0.670014 avg.loss:  3.524785 ETA:   0h49m52s words/sec/thread:  188270 lr:  0.666004 avg.loss:  3.377795 ETA:   0h49m56s% words/sec/thread:  189822 lr:  0.660234 avg.loss:  3.032679 ETA:   0h49m 6s% words/

In [3]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Load dataset into memory
texts, labels = [], []
with open(train_file, "r", encoding="utf-8") as f:
    for line in f:
        label, text = line.strip().split(" ", 1)
        labels.append(label.replace("__label__", ""))
        texts.append(text)

# Split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Predict using FastText
y_pred = []
for line in tqdm(X_test, desc="Predicting"):
    pred_label = model.predict(line)[0][0].replace("__label__", "")
    y_pred.append(pred_label)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))


Predicting: 100%|██████████| 4000/4000 [00:19<00:00, 205.44it/s]


Accuracy: 0.9845
F1 Score: 0.9834090066208308
Precision: 0.9862282339417691
Recall: 0.9845


In [ ]:
import fasttext

model_path = "/home/abhisek/Project/Language_detector_200/FastText_100/lang_detect_model.bin"
model = fasttext.load_model(model_path)

### Testing

In [23]:
test_lines = [
    "The weather is absolutely delightful today, with a gentle, refreshing breeze rustling through the trees and the sun shining brightly, making it perfect for an outdoor stroll. The birds are singing sweetly, and there's a pleasant warmth in the air, inviting everyone to enjoy the beauty of nature. It's truly a splendid day to be outside.",  # English
    "আজকের আবহাওয়া একেবারেই আনন্দদায়ক, গাছের পাতার মধ্যে দিয়ে একটি মৃদু, সতেজ বাতাস বইছে এবং সূর্য উজ্জ্বলভাবে আলোকিত হচ্ছে, যা বাইরের হাঁটার জন্য এটিকে নিখুঁত করে তুলেছে। পাখিরা মিষ্টি সুরে গান গাইছে এবং বাতাসে একটি মনোরম উষ্ণতা রয়েছে, যা সবাইকে প্রকৃতির সৌন্দর্য উপভোগ করার জন্য আমন্ত্রণ জানাচ্ছে। বাইরে থাকার জন্য এটি সত্যিই একটি চমৎকার দিন।",  # Bengali
    "El clima es absolutamente delicioso hoy, con una brisa suave y refrescante susurrando entre los árboles y el sol brillando intensamente, lo que lo hace perfecto para un paseo al aire libre. Los pájaros cantan dulcemente y hay una calidez agradable en el aire, invitando a todos a disfrutar de la belleza de la naturaleza. Es realmente un día espléndido para estar al aire libre.",  # Spanish
    "Das Wetter ist heute absolut herrlich, mit einer sanften, erfrischenden Brise, die durch die Bäume rauscht, und der Sonne, die hell scheint, was es perfekt für einen Spaziergang im Freien macht. Die Vögel singen süßlich, und es liegt eine angenehme Wärme in der Luft, die jeden einlädt, die Schönheit der Natur zu genießen. Es ist wirklich ein prächtiger Tag, um draußen zu sein.",  # German
    "Le temps est absolument magnifique aujourd'hui, avec une brise douce et rafraîchissante qui frôle les arbres et le soleil qui brille de mille feux, ce qui en fait une journée parfaite pour une promenade en plein air. Les oiseaux chantent doucement et il y a une agréable chaleur dans l'air, invitant tout le monde à profiter de la beauté de la nature. C'est vraiment une journée splendide pour être dehors.",  # French
    "O tempo está absolutamente delicioso hoje, com uma brisa suave e refrescante sussurrando entre as árvores e o sol brilhando intensamente, tornando-o perfeito para um passeio ao ar livre. Os pássaros cantam docemente e há um calor agradável no ar, convidando a todos para desfrutar da beleza da natureza. É realmente um dia esplêndido para estar ao ar livre."  # Portuguese
]

for line in test_lines:
    label = model.predict(line)[0][0].replace("__label__", "")
    print(f"Predicted Language: {label}\nSentence: {line}\n")

Predicted Language: English
Sentence: The weather is absolutely delightful today, with a gentle, refreshing breeze rustling through the trees and the sun shining brightly, making it perfect for an outdoor stroll. The birds are singing sweetly, and there's a pleasant warmth in the air, inviting everyone to enjoy the beauty of nature. It's truly a splendid day to be outside.

Predicted Language: Bengali
Sentence: আজকের আবহাওয়া একেবারেই আনন্দদায়ক, গাছের পাতার মধ্যে দিয়ে একটি মৃদু, সতেজ বাতাস বইছে এবং সূর্য উজ্জ্বলভাবে আলোকিত হচ্ছে, যা বাইরের হাঁটার জন্য এটিকে নিখুঁত করে তুলেছে। পাখিরা মিষ্টি সুরে গান গাইছে এবং বাতাসে একটি মনোরম উষ্ণতা রয়েছে, যা সবাইকে প্রকৃতির সৌন্দর্য উপভোগ করার জন্য আমন্ত্রণ জানাচ্ছে। বাইরে থাকার জন্য এটি সত্যিই একটি চমৎকার দিন।

Predicted Language: Spanish
Sentence: El clima es absolutamente delicioso hoy, con una brisa suave y refrescante susurrando entre los árboles y el sol brillando intensamente, lo que lo hace perfecto para un paseo al aire libre. Los pájaros 